## 1.2. Preprocessing weather indexes

In [2]:
library("tidyverse")
library("weathercan")
library("lubridate") # yday(), ymd()
library("Amelia")


Attaching package: 'lubridate'


The following object is masked from 'package:base':

    date


Loading required package: Rcpp

## 
## Amelia II: Multiple Imputation
## (Version 1.7.6, built: 2019-11-24)
## Copyright (C) 2005-2020 James Honaker, Gary King and Matthew Blackwell
## Refer to http://gking.harvard.edu/amelia/ for more information
## 



Load the data frame processed in previous file `1.1_preprocessing-1.ipynb`.

In [2]:
data_df <- read.csv('output/balanced_potato_df.csv')

### Compute 5 years (mean or total) weather indices 

Because the process is long, you can load `data_stations.csv` file and make a left join with `data_df` using "Annee", "LatDD", "LonDD", "DatePlantation", and "DateRecolte" as common features.

In [3]:
data_stations = read_csv("output/weather_stations.csv")

Parsed with column specification:
cols(
  Annee = col_double(),
  LatDD = col_double(),
  LonDD = col_double(),
  DatePlantation = col_date(format = ""),
  DateRecolte = col_date(format = ""),
  station_id = col_double(),
  temp_moy_5years = col_double(),
  prec_tot_5years = col_double(),
  sdi_5years = col_double(),
  gdd_5years = col_double()
)


Custom fonctions 

In [4]:
# shannon diversity index
SDI_f <- function(x) { 
  p <- x/sum(x, na.rm = TRUE)
  SDI <- sum(p * log(p), na.rm = TRUE) / log(length(x))
  return(SDI)
}
# growing degree-days
GDD_f <- function(x, delim = 5) {
  sum(x[x >= delim], na.rm = TRUE)
}

Create a data frame for computations

In [5]:
data_df$DatePlantation <- ymd(data_df$DatePlantation)
data_df$DateRecolte <- ymd(data_df$DateRecolte)

Warning message:
" 1 failed to parse."Warning message:
" 1 failed to parse."

If you've loaded the `weather_stations.csv`, from here, you coud jump to the fith code chaine from the end (insertion into the main table). 

In [7]:
data_stations <- data_df %>%
  distinct(Annee, LatDD, LonDD, DatePlantation, DateRecolte) %>%
  filter(!is.na(Annee))
glimpse(data_stations)

Observations: 323
Variables: 5
$ Annee          <int> 2003, 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2005...
$ LatDD          <dbl> 46.75306, 46.76444, 48.79722, 48.79722, 48.79722, 48...
$ LonDD          <dbl> -72.33861, -72.33083, -72.24250, -72.24250, -72.2425...
$ DatePlantation <date> 2003-05-19, 2004-05-30, 2004-05-31, 2004-06-05, 200...
$ DateRecolte    <date> 2003-09-17, 2004-09-27, 2004-09-20, 2004-09-15, 200...


Impute missing dates if year is recorded

In [8]:
# Check missing dates
data_stations %>%
  filter(is.na(DatePlantation))

Annee,LatDD,LonDD,DatePlantation,DateRecolte
<int>,<dbl>,<dbl>,<date>,<date>
2011,46.02889,-73.65111,NA,NA
1992,48.55000,-71.33333,NA,NA
1993,48.55000,-71.33333,NA,NA
1995,46.10487,-72.24634,NA,1995-09-10
1996,46.10487,-72.24634,NA,1996-09-11
1996,46.10903,-72.38530,NA,1996-09-11
1996,46.45472,-72.70722,NA,1996-09-11
1987,46.45472,-72.70722,NA,1987-09-10
1988,46.45472,-72.70722,NA,1988-09-11


In [9]:
# Check missing dates
data_stations %>%
  filter(is.na(DateRecolte))

Annee,LatDD,LonDD,DatePlantation,DateRecolte
<int>,<dbl>,<dbl>,<date>,<date>
2011,46.02889,-73.65111,NA,NA
1992,48.55000,-71.33333,NA,NA
1993,48.55000,-71.33333,NA,NA


In [10]:
data_stations_imp <- data_stations %>%
                          mutate(DatePlantation_yd = yday(DatePlantation), # yday = year day
                                 DateRecolte_yd = yday(DateRecolte))

data_stations_imp <- amelia(x = data_stations_imp %>%
                         select(Annee, LatDD, LonDD, DatePlantation_yd, 
                                DateRecolte_yd), m=1)$imputations[[1]]

-- Imputation 1 --

  1  2  3



In [11]:
data_stations_imp$Annee <- as.Date(paste0(data_stations_imp$Annee, "-01-01"))

data_stations$DatePlantation <- data_stations_imp$Annee + data_stations_imp$DatePlantation_yd - 1
data_stations$DateRecolte <- data_stations_imp$Annee + data_stations_imp$DateRecolte_yd - 1

In [12]:
data_stations$DatePlantation <- as.Date(data_stations$DatePlantation)
data_stations$DateRecolte <- as.Date(data_stations$DateRecolte)

Load data from Environnement Canada (weathercan)

In [12]:
year_step <- 5 # start 5 years preceeding trial year
data_stations$station_id <- NA # identify weather stations
station_weather <- list() # list of weather stations containning climate data

for (i in 1:nrow(data_stations)) {
  print(paste(i, "/", nrow(data_stations), '...'))
  
  # identify the nearest station. Search for stations by name or location
  all_stations <- stations_search(coords = c(data_stations$LatDD[i], 
                                             data_stations$LonDD[i]),  
                                  interval = "day",
                                  dist = 500)
  annee <- data_stations$Annee[i]
  
  # Available station for nearest year
  closest_station <- all_stations %>%
                        filter(start <= annee - year_step) %>%
                        filter(end >= annee) %>%
                        slice(which.min(distance))

  data_stations$station_id[i] <- closest_station$station_id %>% 
                                      as.character() %>% 
                                      as.numeric()
  
  print(paste("Station id:", data_stations$station_id[i]))
  
  # # Download weather data from Environment Canada of the station identified
  station_weather[[i]] <- weather_dl(station_ids = data_stations$station_id[i], 
                                     start = as.Date(paste0(annee - year_step, "-01-01")),
                                     end = as.Date(paste0(annee, "-01-01")),
                                     interval = "day")
  
  print(paste(i, "/", nrow(data_stations)))
}
# name of the list elements
names(station_weather) <- as.character(data_stations$station_id) 

[1] "1 / 323 ..."
[1] "Station id: 5203"
[1] "1 / 323"
[1] "2 / 323 ..."
[1] "Station id: 5203"
[1] "2 / 323"
[1] "3 / 323 ..."
[1] "Station id: 5929"
[1] "3 / 323"
[1] "4 / 323 ..."
[1] "Station id: 5929"
[1] "4 / 323"
[1] "5 / 323 ..."
[1] "Station id: 5929"
[1] "5 / 323"
[1] "6 / 323 ..."
[1] "Station id: 5929"
[1] "6 / 323"
[1] "7 / 323 ..."
[1] "Station id: 5929"
[1] "7 / 323"
[1] "8 / 323 ..."
[1] "Station id: 5929"
[1] "8 / 323"
[1] "9 / 323 ..."
[1] "Station id: 5203"
[1] "9 / 323"
[1] "10 / 323 ..."
[1] "Station id: 5203"
[1] "10 / 323"
[1] "11 / 323 ..."
[1] "Station id: 5222"
[1] "11 / 323"
[1] "12 / 323 ..."
[1] "Station id: 5222"
[1] "12 / 323"
[1] "13 / 323 ..."
[1] "Station id: 5222"
[1] "13 / 323"
[1] "14 / 323 ..."
[1] "Station id: 5929"
[1] "14 / 323"
[1] "15 / 323 ..."
[1] "Station id: 5255"
[1] "15 / 323"
[1] "16 / 323 ..."
[1] "Station id: 5929"
[1] "16 / 323"
[1] "17 / 323 ..."
[1] "Station id: 5255"
[1] "17 / 323"
[1] "18 / 323 ..."
[1] "Station id: 5255"
[1] "18

Some variables have non-numeric values (spd_max_gust), for stations: 5251

  Replaced all non-numeric entries with NA. Use 'string_as = NULL' to keep as characters (see ?weather_dl).



[1] "24 / 323"
[1] "25 / 323 ..."
[1] "Station id: 5447"
[1] "25 / 323"
[1] "26 / 323 ..."
[1] "Station id: 5255"
[1] "26 / 323"
[1] "27 / 323 ..."
[1] "Station id: 5929"
[1] "27 / 323"
[1] "28 / 323 ..."
[1] "Station id: 5393"
[1] "28 / 323"
[1] "29 / 323 ..."
[1] "Station id: 5222"
[1] "29 / 323"
[1] "30 / 323 ..."
[1] "Station id: 5929"
[1] "30 / 323"
[1] "31 / 323 ..."
[1] "Station id: 5929"
[1] "31 / 323"
[1] "32 / 323 ..."
[1] "Station id: 5929"
[1] "32 / 323"
[1] "33 / 323 ..."
[1] "Station id: 5929"
[1] "33 / 323"
[1] "34 / 323 ..."
[1] "Station id: 5929"
[1] "34 / 323"
[1] "35 / 323 ..."
[1] "Station id: 5929"
[1] "35 / 323"
[1] "36 / 323 ..."
[1] "Station id: 5929"
[1] "36 / 323"
[1] "37 / 323 ..."
[1] "Station id: 5929"
[1] "37 / 323"
[1] "38 / 323 ..."
[1] "Station id: 5929"
[1] "38 / 323"
[1] "39 / 323 ..."
[1] "Station id: 5929"
[1] "39 / 323"
[1] "40 / 323 ..."
[1] "Station id: 5929"
[1] "40 / 323"
[1] "41 / 323 ..."
[1] "Station id: 5929"
[1] "41 / 323"
[1] "42 / 323 ..

There are no data for station 5662, in this time range (1955-01-01 to 1960-01-01), for this interval (day), 
Available Station Data:
# A tibble: 3 x 14
  prov  station_name station_id climate_id WMO_id TC_id   lat   lon  elev tz   
  <chr> <chr>             <int> <fct>       <int> <fct> <dbl> <dbl> <dbl> <chr>
1 QC    BAIE-COMEAU…       5662 7040440        NA NA     49.1 -68.2  21.6 Etc/…
2 QC    BAIE-COMEAU…       5662 7040440        NA NA     49.1 -68.2  21.6 Etc/…
3 QC    BAIE-COMEAU…       5662 7040440        NA NA     49.1 -68.2  21.6 Etc/…
# … with 4 more variables: interval <chr>, start <int>, end <int>,
#   normals <lgl>



[1] "57 / 323"
[1] "58 / 323 ..."
[1] "Station id: 5662"


...

[1] "319 / 323"
[1] "320 / 323 ..."
[1] "Station id: 10843"


Some variables have non-numeric values (spd_max_gust), for stations: 10843

  Replaced all non-numeric entries with NA. Use 'string_as = NULL' to keep as characters (see ?weather_dl).



[1] "320 / 323"
[1] "321 / 323 ..."
[1] "Station id: 5522"
[1] "321 / 323"
[1] "322 / 323 ..."
[1] "Station id: 5237"


Some variables have non-numeric values (spd_max_gust), for stations: 5237

  Replaced all non-numeric entries with NA. Use 'string_as = NULL' to keep as characters (see ?weather_dl).



[1] "322 / 323"
[1] "323 / 323 ..."
[1] "Station id: 5532"
[1] "323 / 323"


Compute indexes for each station

In [15]:
# Initialize columns to record indexes
data_stations$temp_moy_5years <- NA
data_stations$prec_tot_5years <- NA
data_stations$sdi_5years <- NA
data_stations$gdd_5years <- NA

for (i in 1:nrow(data_stations)) {
  starts <- data_stations$DatePlantation[i] - (year_step:1)*365.25 # beginings of seasons
  ends <- data_stations$DateRecolte[i] - (year_step:1)*365.25 # ends of seasons, for previous years
  # Create vectors to record indices from previous years
  temp_moy_j <- rep(NA, length(starts))
  prectot_j <- temp_moy_j
  sdi_j <- temp_moy_j
  gdd_j <- temp_moy_j
  for (j in 1:length(starts)) {
    # Filter the downloaded weather data tables to keep only data measured between the season dates
    season <- station_weather[[i]] %>%
      filter(date >= starts[j]) %>%
      filter(date <= ends[j])
    temp_moy_j[j] <- mean(season$mean_temp, na.rm = TRUE) # average temperature of the season
    prectot_j[j] <- sum(season$total_precip, na.rm = TRUE) # total precipitations of the season
    sdi_j[j] <- SDI_f(season$total_precip) # season SDI
    gdd_j[j] <- GDD_f(season$mean_temp, delim = 5) # season GDD
  }
  data_stations$temp_moy_5years[i] <- mean(temp_moy_j, na.rm = TRUE) # average temperature
  data_stations$prec_tot_5years[i] <- mean(prectot_j, na.rm = TRUE) # average total rainfalls
  data_stations$sdi_5years[i] <- mean(sdi_j, na.rm = TRUE) # average SDI
  data_stations$gdd_5years[i] <- mean(gdd_j, na.rm = TRUE) # average GDD
}

Warning message:
"Unknown or uninitialised column: 'mean_temp'."Warning message in mean.default(season$mean_temp, na.rm = TRUE):
"argument is not numeric or logical: returning NA"Warning message:
"Unknown or uninitialised column: 'total_precip'."Warning message:
"Unknown or uninitialised column: 'total_precip'."Warning message:
"Unknown or uninitialised column: 'mean_temp'."Warning message:
"Unknown or uninitialised column: 'mean_temp'."Warning message in mean.default(season$mean_temp, na.rm = TRUE):
"argument is not numeric or logical: returning NA"Warning message:
"Unknown or uninitialised column: 'total_precip'."Warning message:
"Unknown or uninitialised column: 'total_precip'."Warning message:
"Unknown or uninitialised column: 'mean_temp'."Warning message:
"Unknown or uninitialised column: 'mean_temp'."Warning message in mean.default(season$mean_temp, na.rm = TRUE):
"argument is not numeric or logical: returning NA"Warning message:
"Unknown or uninitialised column: 'total_precip'."

In [16]:
write_csv(x = data_stations, path = "output/weather_stations.csv") # a backup

If you've loaded the `weather_stations.csv`, continue from here. 

Insert into the main table

In [6]:
weath_col <- c(
            'Annee', 'LatDD', 'LonDD', 'DatePlantation', 'DateRecolte', 
            'temp_moy_5years', 'prec_tot_5years', 'sdi_5years', 'gdd_5years'
            )

In [7]:
data.frame(weath_col, weath_col %in% colnames(data_df))

weath_col,weath_col..in..colnames.data_df.
<fct>,<lgl>
Annee,TRUE
LatDD,TRUE
LonDD,TRUE
DatePlantation,TRUE
DateRecolte,TRUE
temp_moy_5years,FALSE
prec_tot_5years,FALSE
sdi_5years,FALSE
gdd_5years,FALSE


In [8]:
data_df <- left_join(data_df, data_stations[weath_col], 
                     by = c("Annee", "LatDD", "LonDD", "DatePlantation", "DateRecolte"))

In [9]:
wcol <- c('temp_moy_5years', 'prec_tot_5years', 'sdi_5years', 'gdd_5years')
data.frame(wcol, wcol %in% colnames(data_df))

wcol,wcol..in..colnames.data_df.
<fct>,<lgl>
temp_moy_5years,TRUE
prec_tot_5years,TRUE
sdi_5years,TRUE
gdd_5years,TRUE


In [10]:
write_csv(data_df, "output/pr_potato_df.csv") # backup